In [1]:
import pandas as pd
import numpy as np
import pyodbc
import json
import string
import pickle
from jieba import lcut
from collections import Counter
from statistics import mean
from scipy.sparse import csr_matrix
from tqdm import tqdm
import time
import pkuseg
from gensim import corpora
#import langdetect 
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold # import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

from sentence_transformers import models, SentenceTransformer
import numpy as np
import string
pd.set_option('display.max_rows', 50000)

random_seed = 751031
np.random.seed(random_seed)

In [2]:
#function zone
def Bulid_dict(message_list):
# 總分詞
    ls_of_words = [lcut(text) for text in message_list]
    # 構造詞典
    dictionary = corpora.Dictionary(ls_of_words)
    dt = dictionary.token2id
    length = len(dt)
    return dt, length

def vectorize(ls_of_words, dt):
    length = len(dt)
    # 句向量
    ls_of_wid = []
    for words in ls_of_words:#ls_of_words 
        vector = [0] * length
        for word in words:
            try:
                vector[dt[word]] += 1
            except:
                continue
        ls_of_wid.append(vector) 
    return ls_of_wid

def add_feature(sr):
    num_only = int(sr.isdigit())
    abc_only = int(sr.encode( 'UTF-8').isalpha())
    #numorabc_only = int(sr.isalnum())
    def is_contains_chinese(strs):
        for _char in strs:
            if '\u4e00' <= _char <= '\u9fa5':
                return 1
        return 0
 
    #检验是否全是中文字符
    def is_all_chinese(strs):
        for _char in strs:
            if not '\u4e00' <= _char <= '\u9fa5':
                return 0
        return 1
    #allchinese = is_all_chinese(sr)
    #containchinese = is_all_chinese(sr)   
    return [num_only, abc_only]


def remove_punctuation(s): 
    punctuation = '''''：，？表情\n!！()-[]{};:'"\,<>./[email protected]#$%^&*_~'''+ string.digits+string.whitespace+ string.punctuation
    #'''''\n!()-[]{};:'"\,<>./[email protected]#$%^&*_~！（）－［］｛｝；：’＂＼，＜＞．／＃＄％︿＆＊＿～'''
    my_str = s 
    no_punct = "" 
    for char in my_str: 
        if char not in punctuation: 
            no_punct += char 
    return(no_punct) 

def change_string(articut, text):
    if text == '':
        tmp = ''
    else:
        result = articut.parse(text)
        content = articut.getContentWordLIST(result)
        tmp = ''
        for sentense in content:
            for word in sentense:
                tmp += word[-1]
    return tmp

## 建構一個總辭典

In [5]:
data_modify = pd.read_excel("C://Users//user//Desktop//project//up//input//input_v5.xlsx", index_col = None, engine='openpyxl', header=0)
data_modify = data_modify[['message', '正确意图']]
data_modify = data_modify[~data_modify.loc[:, '正确意图'].isna()]
data_modify.columns = ['message', 'label']

In [6]:
change_dict = {'到帐问题':0, '提款卖币问题':1, '存款买币问题':2, '银行卡问题':3, '操作问题':4, '订单取消问题':5, '非业务意图':6}
change_dict_verse = {v: k for k, v in change_dict.items()}
data_modify = data_modify[data_modify.label.isin(list(change_dict.keys()))].reset_index(drop=True)
print(data_modify.shape)
data_modify.loc[:, 'message'] = data_modify.loc[:, 'message'].apply(remove_punctuation)
data_modify.loc[:, 'message'] = [i.upper()  for i in data_modify.message]

(4675, 2)


In [7]:
pku_seg = pkuseg.pkuseg()

In [8]:
def Bulid_dict(message_list):
# 總分詞
    ls_of_words = [pku_seg.cut(text) for text in message_list]
    # 構造詞典
    dictionary = corpora.Dictionary(ls_of_words)
    dt = dictionary.token2id
    length = len(dt)
    return dt, length

In [9]:
dt, length = Bulid_dict(data_modify.message)

In [10]:
length

1559

# 预处理

In [11]:
CountVector = CountVectorizer()
CountVectorizer_fit = CountVector.fit_transform(data_modify.message)
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)
tv.fit(data_modify.message)
tv_fit = tv.transform(data_modify.message)
#pickle.dump(tv, open('tv.pkl', 'wb'))
#tv = pickle.load(open('tv.pkl', 'rb'))

In [ ]:
pickle.dump(tv, open('tv.pkl', 'wb'))
#tv = pickle.load(open('tv.pkl', 'rb'))

In [12]:
'''
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data_modify.message)
xtrain_tfidf = tfidf_vect.transform(data_modify.message)
fuzz.token_sort_ratio("我想要存款", "我想存款")
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")
train_list2 = []
for i in train_list:
    train_list2.append([x for x in i if x not in stop_words])
'''
train_list = [pku_seg.cut(text) for text in data_modify.message]#[lcut(text) for text in data_modify.message]
train_list_vec = vectorize(train_list, dt)

In [13]:
'''
#label_dict = list(change_dict.keys())
#data_modify['cut_count'] = data_modify['message'].apply(lambda x: len(lcut(x)))
#data_modify['cut_density'] = data_modify['cut_count'] / (data_modify['word_count']+1)
#data_modify['title_word_count'] = data_modify['message'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
#data_modify['v0'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[0])/100)
#data_modify['v1'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[1])/100)
#data_modify['v2'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[2])/100)
#data_modify['v3'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[3])/100)
#data_modify['v4'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[4])/100)
#data_modify['v5'] = data_modify['message'].apply(lambda x: fuzz.token_sort_ratio(x, label_dict[5])/100)
#data_modify['v0_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[0])/100)
#data_modify['v1_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[1])/100)
#data_modify['v2_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[2])/100)
#data_modify['v3_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[3])/100)
#data_modify['v4_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[4])/100)
#data_modify['v5_1'] = data_modify['message'].apply(lambda x: fuzz.ratio(x, label_dict[5])/100)
'''
data_modify['char_count'] = data_modify['message'].apply(len)
data_modify['word_count'] = data_modify['message'].apply(lambda x: len(x.split()))
data_modify['word_density'] = data_modify['char_count'] / (data_modify['word_count']+1)
data_modify['punctuation_count'] = data_modify['message'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))

add_feature_ = data_modify[['word_count', 'word_density', 'punctuation_count']].values
add_feature_isin = np.array([add_feature(i) for i in data_modify.message])

In [14]:
train_data = np.c_[np.array(train_list_vec), 
                   add_feature_isin, 
                   tv_fit.toarray(),
                   add_feature_]

In [15]:
train_data.shape

(4675, 5488)

In [16]:
import os
os.getcwd()

'C:\\Users\\user\\Desktop\\project\\up\\Train'

In [17]:
maxminscaler = MinMaxScaler()
maxminscaler.fit(train_data)
train_data = maxminscaler.transform(train_data)

In [18]:
#pickle.dump(maxminscaler, open('maxminscaler.pkl', 'wb'))

In [19]:
label_change = [change_dict[i] for i in data_modify.label]
data_modify.loc[:, 'label'] = label_change

In [20]:
# 資料切分
train_y, test_y, train_x, test_x = train_test_split(data_modify.label, 
                                                    train_data,#train_list_vec, 
                                                    test_size=0.1, 
                                                    random_state=random_seed)
train_y = train_y.reset_index(drop=True)
del(train_data)

In [ ]:
label_change = [change_dict[i] for i in data_modify.label]
data_modify.loc[:, 'label'] = label_change

In [ ]:
train_x_oversameple = np.zeros((0,train_x.shape[1]), float)
train_y_oversameple = np.zeros((0,), float)

max_sample = train_y.shape[0]#Counter(train_y).most_common(1)[0][1]
np.random.seed(random_seed)
for i in train_y.unique():
    ta = list(np.random.choice(list(train_y[train_y==i].index), max_sample))
    train_y_oversameple = np.append(train_y_oversameple, train_y[ta], axis=0)
    train_x_oversameple = np.append(train_x_oversameple, train_x[ta], axis=0)

In [21]:
n_fold = 5
kf = KFold(n_splits=n_fold, random_state=random_seed, shuffle=True)
accuracy_list = []
for train_index, valid_index in kf.split(train_x):
    #print(valid_index)
    X_train, X_test = train_x[train_index], train_x[valid_index]
    y_train, y_test = train_y[train_index], train_y[valid_index]
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    max_sample = X_train.shape[0]#Counter(train_y).most_common(1)[0][1]
    X_train_oversameple = np.zeros((0, X_train.shape[1]), float)
    y_train_oversameple = np.zeros((0,), float)
    np.random.seed(random_seed)
    for i in range(7):
        ta = list(np.random.choice(list(y_train[y_train==i].index), max_sample))
        y_train_oversameple = np.append(y_train_oversameple, y_train[ta], axis=0)
        X_train_oversameple = np.append(X_train_oversameple, X_train[ta], axis=0)    
    classifier = MultinomialNB()  # 樸素貝葉斯分類器
    classifier.fit(X_train_oversameple, y_train_oversameple)
    # 模型測評
    score = classifier.score(X_test, y_test)
    accuracy_list.append(score)
    #del(X_train, X_test, y_train, y_test, classifier)
    
print('{k}_fold cross valid accuracy mean:{mean}, max:{max}, min:{min}'.format(k=n_fold, 
                                                                               mean=mean(accuracy_list),
                                                                               max=max(accuracy_list),
                                                                               min=min(accuracy_list)))

5_fold cross valid accuracy mean:0.602566224464146, max:0.6111771700356718, min:0.5945303210463734


In [ ]:
# 貝葉斯模型訓練
classifier = MultinomialNB()  # 樸素貝葉斯分類器
classifier.fit(train_x_oversameple, train_y_oversameple)

# 模型測評
score = classifier.score(test_x, test_y)
print('Test set Accuracy：'+ str(round(score*100, 2))+'%')

In [ ]:
classifier.coef_.shape

In [ ]:
'''
#save model and load the model
filename = 'C://Users//user//Desktop//aicode-master//textLearn//models//nb//nb.sav'
pickle.dump(classifier, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))
'''

## Model -logistic regression

In [22]:
n_fold = 5
random_seed = 751031
kf = KFold(n_splits=n_fold, random_state=random_seed, shuffle=True)
accuracy_list = []
for train_index, valid_index in kf.split(train_x):
    #print(valid_index)
    X_train, X_test = train_x[train_index], train_x[valid_index]
    y_train, y_test = train_y[train_index], train_y[valid_index]
    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    #for i in range(7):
        #x = list(y_test).count(i)
        #ratio = x/ y_test.shape[0]
        #print(change_dict_verse[i]+ str(round(ratio*100,2))+ '%')
    #print("----------------------------")
    max_sample = X_train.shape[0]#Counter(train_y).most_common(1)[0][1]
    X_train_oversameple = np.zeros((0, X_train.shape[1]), float)
    y_train_oversameple = np.zeros((0,), float)
    np.random.seed(random_seed)
    for i in range(7):
        ta = list(np.random.choice(list(y_train[y_train==i].index), max_sample))
        y_train_oversameple = np.append(y_train_oversameple, y_train[ta], axis=0)
        X_train_oversameple = np.append(X_train_oversameple, X_train[ta], axis=0)    
    lr = LogisticRegression(solver='saga',
                            multi_class='multinomial',
                            C=1,
                            penalty='l1',
                            fit_intercept=True,
                            max_iter=5000,
                            random_state=4180,
                            class_weight='balanced',
                            n_jobs=-1)
    lr.fit(csr_matrix((X_train_oversameple), dtype=float), 
           y_train_oversameple)
    # 模型測評
    score = lr.score(csr_matrix((X_test), dtype=float), 
                     y_test)
    accuracy_list.append(score)
    print('-----')
    #del(X_train, X_test, y_train, y_test, classifier)
    
print('{k}_fold cross valid accuracy mean:{mean}, max:{max}, min:{min}'.format(k=n_fold, 
                                                                               mean=mean(accuracy_list),
                                                                               max=max(accuracy_list),
                                                                               min=min(accuracy_list)))

-----
-----
-----
-----
-----
5_fold cross valid accuracy mean:0.8202959377056496, max:0.8396674584323041, min:0.8061831153388823


In [ ]:
X_train_oversameple.shape

In [ ]:
s = time.time()
lr = LogisticRegression(solver='saga',
                        multi_class='multinomial',
                        C=1,
                        penalty='l1',
                        fit_intercept=True,
                        max_iter=5000,
                        random_state=4180,
                        class_weight='balanced',
                        n_jobs=-1)
lr.fit(csr_matrix((train_x_oversameple), dtype=float),
       train_y_oversameple)
accuracy = lr.score(csr_matrix((test_x), dtype=float), 
                    test_y)
print('LR - Test set Accuracy：'+str(round(accuracy*100, 2))+'%')
print(time.time()-s)

In [ ]:
PRED_Y = lr.predict(csr_matrix(test_x, dtype=float))

In [ ]:
for i in range(7):
    x = list(test_y).count(i)
    ratio = x/ test_y.shape[0]
    print(str(x)+ '_'+ str(i)+  '_'+ str(round(ratio*100,2))+ '%')

In [ ]:
for i in range(7):
    x = list(PRED_Y).count(i)
    ratio = x/ test_y.shape[0]
    print(str(x)+ '_'+ str(i)+  '_'+ str(round(ratio*100,2))+ '%')

In [ ]:
#save model and load the model
filename = 'C://Users//user//Desktop//LR_UP'
pickle.dump(lr, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))

In [ ]:
'''
#save model and load the model
filename = 'C://Users//user//Desktop//aicode-master//textLearn//models//nb//nb.sav'
pickle.dump(classifier, open(filename, 'wb'))
#classifier = pickle.load(open(filename, 'rb'))
'''

# pred result (单一)

In [ ]:
''''''
strs = '游戏没法充值'
test_1 = [lcut(strs)]
dict_key = list(dt.keys())
tmp = []
for i in test_1[0]:
    if i in dict_key:
        tmp.append(i)
test_1 = [tmp]


strs_df = pd.DataFrame({'message':[strs]})
strs_df['char_count'] = strs_df['message'].apply(len)
strs_df['word_count'] = strs_df['message'].apply(lambda x: len(x.split()))
strs_df['word_density'] = strs_df['char_count'] / (strs_df['word_count']+1)
strs_df['punctuation_count'] = strs_df['message'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))


add_feature_test = strs_df[['word_count', 'word_density', 'punctuation_count']].values
tran = tv.transform(pd.Series(strs))

#tfidf = tfidf_vect.transform(pd.Series(strs))
test_1 = vectorize(test_1, dt)
test_1 = np.c_[np.array(test_1), np.array(add_feature(strs)).reshape((1, 2)), tran.toarray(), add_feature_test]

maxminscaler = pickle.load(open('maxminscaler.pkl', 'rb'))
test_1 = maxminscaler.transform(test_1)
#test_1 = np.c_[np.array(test_1), np.array(add_feature(strs)).reshape((1, 2)), add_feature_test]

In [ ]:
y2 = lr.predict(test_1)
change_dict_verse[y2[0]]

In [ ]:
#test_1 = np.array(test_1)
y = classifier.predict(test_1)
change_dict_verse[y[0]]

In [ ]:
n_fold = 5
kf = KFold(n_splits=n_fold, random_state=1234, shuffle=True)
result_prob_list = np.zeros((test_1.shape[0], 7), dtype=float)
for train_index, valid_index in kf.split(train_x_oversameple):
    #print(valid_index)
    X_train, X_test = train_x_oversameple[train_index], train_x_oversameple[valid_index]
    y_train, y_test = train_y_oversameple[train_index], train_y_oversameple[valid_index]
    classifier = MultinomialNB()  # 樸素貝葉斯分類器
    classifier.fit(X_train, y_train)
    result_prob_list += classifier.predict_proba(test_1)
    del(X_train, X_test, y_train, y_test, classifier)
change_dict_verse[np.argmax(result_prob_list)]

In [ ]:
'''
for i in data_modify.label.unique():
    #print(i)
    x = list(data_modify.label).count(i)
    ratio = x/ data_modify.shape[0]
    print(change_dict_verse[i]+ str(round(ratio*100,2))+ '%')
'''

In [ ]:
data_modify.label.hist()

In [ ]:
data_modify.label.unique()

In [ ]:
change_dict_verse

In [ ]:
'usdt我要摸報'.upper() 